In [29]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import os


In [30]:
load_dotenv()

True

In [31]:
GROQ=os.getenv("GROQ_API_KEY")

In [32]:
from langchain_groq import ChatGroq



In [ ]:
model = ChatGroq(groq_api_key=GROQ, model="llama-3.3-70b-versatile")

In [34]:
class BlogState(TypedDict):

    title : str
    outline : str
    content : str
    

In [35]:
def create_outline(state: BlogState) ->BlogState:

    title = state["title"]
    

    prompt = f'Generate a detailed outline for a blog on the topic - {title}'

    outline = model.invoke(prompt).content

    state['outline'] = outline
    return state


In [36]:
def create_blog(state: BlogState) ->BlogState:

    title= state["title"]
    outline = state["outline"]

    # call llm
    prompt = f'Write a detailed blog in the title {title} using the following outline {outline}'

    content = model.invoke(prompt).content
    state["content"]= content

    return state

In [37]:
graph = StateGraph(BlogState)




### node
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)



## edges

graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)


## compile

workflow=graph.compile()


In [38]:
initial_state = { "title": "Rise of AI in India"}

final_state = workflow.invoke(initial_state)

print(final_state)

BadRequestError: Error code: 400 - {'error': {'message': 'The model `whisper-large-v3` does not support chat completions', 'type': 'invalid_request_error'}}